In [2]:
import pandas as pd
from PIL import Image
import numpy as np
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from joblib import Parallel, delayed
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


df = pd.read_csv('data/labelled/labelled_data.csv')

# Lista para armazenar os dados
data = []
labels = []

batch_size = 32
img_height = 180
img_width = 180
data_dir='data/cuted/'

2024-03-25 14:56:36.540600: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-25 14:56:36.728773: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-25 14:56:37.417566: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-25 14:56:39.888348: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 602 files belonging to 6 classes.


Using 482 files for training.


## Pré-processamento de imagens: 
Você pode adicionar mais etapas de pré-processamento de imagens, como normalização, aumentação de dados, etc.

## Paralelização: 
A leitura e o processamento de imagens podem ser paralelizados usando multiprocessing ou joblib para acelerar o processo.

## Ajuste de hiperparâmetros: 
Você pode usar técnicas como GridSearchCV ou RandomizedSearchCV para encontrar os melhores hiperparâmetros para seus modelos.

## Validação cruzada: 
Em vez de dividir os dados apenas uma vez, você pode usar a validação cruzada para obter uma estimativa mais robusta do desempenho do modelo.

## Verificação de overfitting: 
Verifique se o modelo está overfitting nos dados de treinamento. Se estiver, você pode tentar adicionar regularização ou usar um modelo mais simples.

In [4]:
# class ProcessImage:
#     def __init__(self, df):
#         self.df = df

#     def process_single_image(self, row):
#         img = Image.open(row['image_path'])
#         img = img.resize((260, 260))
#         img_array = np.array(img).flatten()
#         return img_array, row['label']

#     def process_images(self):
#         results = Parallel(n_jobs=-1)(delayed(self.process_single_image)(row) for _, row in self.df.iterrows())
#         data, labels = zip(*results)
#         return data, labels


# class TrainModel:
#     def __init__(self, data, labels):
#         self.data = data
#         self.labels = labels

#     def train(self):
#         X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

#         # Criar e treinar os modelos com validação cruzada e normalização
#         mlp = MLPClassifier(random_state=42)
#         rf = RandomForestClassifier(random_state=42)

#         print("MLP Cross-Validation Score:")
#         print(np.mean(cross_val_score(mlp, X_train, y_train, cv=5)))

#         print("RandomForest Cross-Validation Score:")
#         print(np.mean(cross_val_score(rf, X_train, y_train, cv=5)))

In [6]:
# Iterar sobre cada linha do DataFrame
for index, row in df.iterrows():
    # Abrir a imagem
    img = Image.open(row['image_path'])
    img = img.resize((260, 260))
    
    # Converter a imagem em um array numpy e achatá-la
    img_array = np.array(img).flatten()
    print(img_array)
    
    # Adicionar o array de imagem e o rótulo à lista de dados
    data.append(img_array)
    labels.append(row['label'])
    np.savetxt(f'image_{index}.txt', img_array, fmt='%d')

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Treinar um modelo MLP
mlp = MLPClassifier(random_state=42)
mlp.fit(X_train, y_train)
mlp_predictions = mlp.predict(X_test)
mlp_accuracy = accuracy_score(y_test, mlp_predictions)
print(f"MLP Accuracy: {mlp_accuracy}")

# Treinar um modelo Random Forest
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
rf_predictions = rf.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print(f"Random Forest Accuracy: {rf_accuracy}")

[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]
[255 255 255 ... 255 255 255]


KeyboardInterrupt: 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

# Função para processar uma única imagem
def process_image(row):
    img = Image.open(row['image_path'])
    img = img.resize((260, 260))
    img_array = np.array(img).flatten()
    return img_array, row['label']

# Paralelizar a leitura e o processamento de imagens
results = Parallel(n_jobs=-1)(delayed(process_image)(row) for _, row in df.iterrows())
data, labels = zip(*results)

# Dividir os dados
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Criar e treinar os modelos com validação cruzada e normalização
mlp = make_pipeline(StandardScaler(), MLPClassifier(random_state=42))
rf = make_pipeline(StandardScaler(), RandomForestClassifier(random_state=42))

print("MLP Cross-Validation Score:")
print(np.mean(cross_val_score(mlp, X_train, y_train, cv=5)))

print("RandomForest Cross-Validation Score:")
print(np.mean(cross_val_score(rf, X_train, y_train, cv=5)))

KeyboardInterrupt: 